In [81]:
import pandas as pd
import os
from parse import *
from scipy.stats import wilcoxon

In [82]:
%run prepareStats.py

In [83]:
measures   = ["ndcg","infNDCG", "Rprec", "P_10"]

In [84]:
statspath = "/Users/faessler/nfshome/Research/sigir/recall/stats_recall_2018/"

statfiles = sorted(list(filter(lambda f: f.endswith(".csv.gz"), os.listdir(statspath))))
runstatsmap = {}
for stat in statfiles:
    df = prepareStats(statspath+stat)
    run = stat.replace("OFFICIAL_", "").replace(".csv.gz", "")
    runstatsmap[run] = df
allstats = pd.concat(runstatsmap)
allstats.index.names = ["run", "Topic"]
allstats = allstats[measures]
allstats = allstats.reset_index()

In [85]:
filenameformat = "{expansions}--mmm:{querymethod}-op:{op}-wr:{wc}-sl:{slop}"
df = allstats
parses = []
for run in df["run"]:
    parses.append(parse(filenameformat, run).named)
indexdf = pd.DataFrame(parses,columns=parses[0].keys())
index = pd.Index(range(len(df)))
df.index = index
df = pd.concat([indexdf,df],axis=1)
#df = df.drop("run",axis=1)

# Are the results with AND and PHRASE the same?
Answer: No.

In [86]:
phrasevalues = df.query("querymethod == 'phrase' and slop == '10'")["infNDCG"]
andvalues    = df.query("op == 'AND' and querymethod == 'best_fields'")["infNDCG"]

print(wilcoxon( andvalues, phrasevalues))


WilcoxonResult(statistic=1088511.0, pvalue=8.077592388141554e-83)
